In [1]:
import warnings, random, os, sys, tqdm, time
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
from scipy.spatial import distance
from scipy.stats import pearsonr

import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_tabnet.tab_model import TabNetRegressor
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline



In [142]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def metric2(y_true, y_preds):
    y_preds = np.clip(y_preds, 0+1e-15, 1-1e-15)
    err = - (y_true * np.log(y_preds) + (1-y_true) * np.log(1-y_preds))
    return err.sum() / y_true.size

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
seed_everything(42)
        
    
def make_scaler(flag, seed):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    elif flag == "gauss":
        return GaussRankScaler()
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
def special_pearsonr(X, Y):
    pearsonr_value = pearsonr(X, Y)
    if pearsonr_value[0] < 0:
        return abs(pearsonr_value[0])
    else:
        return 1 - pearsonr_value[0]    

seeds = [0, 1, 2, 3, 4, 5, 6]
SCALE = "quantile"

In [16]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
pub_test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
drug_df = pd.read_csv("../../../Data/Raw/train_drug.csv")#

train_df = train_df.merge(drug_df, on="sig_id")

y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)
y = y.merge(drug_df, on='sig_id', how='left') #

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
BIOS = GENES + CELLS


SCORED_MOAS = [col for col in y.columns if col != "sig_id" and col != "drug_id"]#
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS


TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
pub_test_df["time_dose"] = pub_test_df["cp_time"].astype(str) + " * " + pub_test_df["cp_dose"]

# remove cp_type = ctl_vehicle
#mask = train_df["cp_type"] != "ctl_vehicle"
#train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
#test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
#pub_test_df = pub_test_df[pub_test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
#y_nonv = y[mask].reset_index(drop=True)#





In [17]:
train_preds = pd.read_csv("train_preds.csv")
train_preds = train_preds.merge(drug_df, on="sig_id")
vc = train_preds["drug_id"].value_counts()
print(vc[:20])

cacb2b860    1866
87d714366     718
9f80f3f77     246
8b87a7a83     203
5628cb3ee     202
d08af5d4b     196
292ab2c28     194
d50f18348     186
d1b47f29d     178
67c879e79      19
d488d031d      18
52d1e6f43      18
83a9ea167      18
6b8b675cc      14
a7c2673c1      14
11f66c124      14
5d9bb0ebe      14
30aa2f709      14
1a52478dc      14
ab50adc64      13
Name: drug_id, dtype: int64


In [18]:
train_df["tag"] = ""
for col in [col for col in SCORED_MOAS]:
    train_df["tag"] += y[col].astype("str")
print(train_df["tag"].unique().shape)

moa_dict = {}
for tag in train_df["tag"].unique():
    name = ""
    for i, col in enumerate(SCORED_MOAS):
        if tag[i] == '1':
            name += col
            name += " : "
    if len(name) == 0:
        name += "dont have moa : "
    name = name[:-3]
    moa_dict[tag] = name
train_df["tag"] = train_df["tag"].map(moa_dict)


(328,)


In [143]:
print(metric2(y.drop(columns=["drug_id", "sig_id"]).values, train_preds.drop(columns=["drug_id", "sig_id"]).values))
print(metric(y.drop(columns=["drug_id", "sig_id"]).values, train_preds.drop(columns=["drug_id", "sig_id"]).values))

0.015613774063774614
0.015613774063774623


In [224]:
def pospro(tr_y, preds_):
    """
    tr_y : moas + sig_id  + drug_id
    preds : moas + sig_id
    """
    preds = preds_.copy()
    vc = tr_y["drug_id"].value_counts()
    MOAS = [col for col in tr_y.columns if col != "drug_id" and col != "sig_id"]
    thres = [0.99, 0.9, 0.99, 0.99, 0.99, 0.9, 0.7, 0.9]
    maxs = []
    y_pos = [
        
    ]

    
    for i in [1, 2, 3, 4, 5, 6, 7, 8]:
        d = vc.index[i]
        n = vc[i]
        print(d, n)
        temp = tr_y[tr_y["drug_id"] == d][MOAS].iloc[0]
        y_pos = temp[MOAS].values
        moas = temp[temp > 0].index.to_list()
        other = [m for m in MOAS if m not in moas]
        if len(moas) == 0:
            continue
        
        temp = preds.copy()
        temp["max"] = temp[other].max(axis=1)
        for m in moas:
            temp = temp[temp[m] > thres[i-1]]
        temp = temp[temp["max"] < 0.1]
        print(temp[temp["drug_id"] != d].shape[0], temp[temp["drug_id"] == d].shape[0])

        idxs = temp.index
        preds.loc[idxs, MOAS] = y_pos
        
    return preds
        
        
train_preds2 = pospro(y, train_preds)       

In [323]:
def pospro2(tr_y, preds_):
    """
    tr_y : moas + sig_id  + drug_id
    preds : moas + sig_id
    """
    preds = preds_.copy()
    vc = tr_y["drug_id"].value_counts()
    MOAS = [col for col in tr_y.columns if col != "drug_id" and col != "sig_id"]
    thres = [0.99, 0.9, 0.99, 0.99, 0.99, 0.9, 0.7, 0.9]
    co_moass = [
        ['proteasome_inhibitor','ikk_inhibitor','ppar_receptor_agonist','nitric_oxide_production_inhibitor','nrf2_activator','nfkb_inhibitor','apoptosis_stimulant','bcl_inhibitor','topoisomerase_inhibitor'],
        ['hmgcr_inhibitor','phosphodiesterase_inhibitor','lxr_agonist','bacterial_50s_ribosomal_subunit_inhibitor','bacterial_cell_wall_synthesis_inhibitor','bacterial_30s_ribosomal_subunit_inhibitor','serotonin_receptor_antagonist'],
        ['cdk_inhibitor','gsk_inhibitor','jak_inhibitor','flt3_inhibitor','topoisomerase_inhibitor','rna_polymerase_inhibitor','focal_adhesion_kinase_inhibitor','lipoxygenase_inhibitor','pdgfr_inhibitor','pkc_inhibitor','bacterial_cell_wall_synthesis_inhibitor'],
        ['egfr_inhibitor','btk_inhibitor','bcr-abl_inhibitor','vegfr_inhibitor','tyrosine_kinase_inhibitor','src_inhibitor','erbb2_inhibitor'],
        ['tubulin_inhibitor','src_inhibitor','opioid_receptor_agonist','atpase_inhibitor','tyrosine_kinase_inhibitor','trpv_agonist','acetylcholine_receptor_agonist','dna_inhibitor','apoptosis_stimulant'],
        [],
        ['vegfr_inhibitor','pdgfr_inhibitor','raf_inhibitor','kit_inhibitor','flt3_inhibitor','fgfr_inhibitor'],
        ['raf_inhibitor','pdgfr_inhibitor','vegfr_inhibitor','kit_inhibitor','fgfr_inhibitor'],
        
    
    ]
    
    for i in [1, 2, 3, 4, 5, 6, 7, 8]:
        d = vc.index[i]
        n = vc[i]
        
        temp = tr_y[tr_y["drug_id"] == d][MOAS].iloc[0]

        moas = temp[temp > 0].index.to_list()
        co_moas = co_moass[i-1]
        for moa in moas:
            temp = tr_y[tr_y[moa] == 1][MOAS].sum()
            co_moa = temp[temp != 0].index.to_list()
            co_moas += co_moa
        
        co_moas = [m for m in co_moas if m not in moas]
        co_moas = list(set(co_moas))
        other = [m for m in MOAS if m not in moas and m not in co_moas]

        if len(moas) == 0:
            continue
        
        temp = preds.copy()
        temp["max"] = temp[other].max(axis=1)
        for m in moas:
            temp = temp[temp[m] > thres[i-1]] # thres[i-1]
        temp = temp[temp["max"] < 0.1]
        

        idxs = temp.index
        #preds.loc[idxs, moas] = 1
        #preds.loc[idxs, co_moas] = 0
        preds.loc[idxs, other] = 0


        
    return preds
train_preds3 = pospro2(y, train_preds)

In [324]:
print(metric2(y.drop(columns=["drug_id", "sig_id"]).values, train_preds.drop(columns=["drug_id", "sig_id"]).values))
print(metric2(y.drop(columns=["drug_id", "sig_id"]).values, train_preds2.drop(columns=["drug_id", "sig_id"]).values))
print(metric2(y.drop(columns=["drug_id", "sig_id"]).values, train_preds3.drop(columns=["drug_id", "sig_id"]).values))

0.015613774063774614
0.01557034066088058
0.015576599288705159


In [267]:
idxs = [586, 2870, 4461, 8938, 15170, 16173, 19169, 19313, 19364, 20150, 20477, 20915, 21117, 22035]



temp = y.iloc[idxs].sum()[SCORED_MOAS].sort_values(ascending=False)
temp[temp != 0].index.to_list()

['raf_inhibitor',
 'pdgfr_inhibitor',
 'vegfr_inhibitor',
 'kit_inhibitor',
 'fgfr_inhibitor']

0.1332234420946666
0.14278133639267512
0.9790079849106924
0.21562966278621126
0.9959911533764432
0.012449626877371752
0.9280971884727478
0.46638458541461403
0.6206574354852951
0.6578923632124706
0.8113151959010532
0.08642809582358625
0.9944726739610944
0.006189066218212247
0.35646181114550146
0.017500131457511867
0.03923332731106451
0.2603853685515267
0.6525771064417702
0.047823516651988016
0.14842625654169495
0.1008125747154866
0.9997499670301164
0.9959911533764432
0.03923332731106451
0.9999910593032836
0.46638458541461403
0.548285927024803
0.9999802964074272
0.9912295171192714
0.06054566347406112
0.3869440449135644
0.06368994420128209
0.6535135592733111
0.00282812029555706
0.4993069392762014
0.548285927024803
0.1008125747154866
0.9978450451578412
0.1332234420946666
0.9790079849106924
0.012449626877371752
0.6535135592733111
0.06817394721188716
0.548285927024803
0.021002640681607388
0.3176871373185089
0.07461509294807911
0.8915771075657436
0.22548027815563335
0.8017163021223884
0.99915

In [326]:
train_preds

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [332]:
sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv", index_col="sig_id")
sub_df

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,re

In [333]:
sub_df = sub_df.reset_index()
sub_df

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [335]:
D1_24 = {
    11053:29.32188365,
    7898:38.32122045,
    3139:69.80596651,
    12147:52.59766894,
    19015:35.09438752,
    13507:51.30393357,
    5106:31.23270931,
    1036:48.51246837,
}
D1_48 = {
    19541:32.81430755,
    5150:40.26684705,
    7983:65.63935697,
    12911:56.32725683,
    22073:34.5371754,
    22923:62.75542217,
    23116:30.03966032,
    17188:52.25682962,
}
D1_72 = {
    9289:32.5153389,
    19870:36.4486593,
    5954:61.36761809,
    6657:61.02059108,
    2659:34.26900523,
    6618:74.80773455,
    22746:25.55613713,
    13970:29.35582608,
    15314:43.98533905,
}

D2_24 = {
    17608:32.9819429,
    4612:52.08722652,
    17419:35.41609046,
    14485:66.24034192,
    14102:55.76277433,
    18273:31.85946914,
    22754:67.42243075,
    12561:26.22151075,
    9874:31.2302477,
    17611:36.09726524,
    7592:53.94529639,
}
D2_48 = {
    7061:31.68985488,
    18509:37.59451255,
    19921:68.1128963,
    13303:64.76175793,
    4966:33.98200843,
    14209:26.79777018,
    17862:32.5388908,
    255:53.06485724,
    
}

D2_72 = {
    17762:37.02551093,
    23755:39.3028134,
    20553:61.86157087,
    1475:66.46391752,
    5655:35.5196236,
    7036:37.44239854,
    20608:49.21400435,
}

In [354]:
from scipy.spatial.distance import euclidean
#train_df = pd.read_csv("../input/lish-moa/train_features.csv")
train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
#test_df = pd.read_csv("../input/lish-moa/test_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
#train_y = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
train_y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
MOAS = [col for col in train_y.columns if col != "sig_id"]
GENES = [col for col in train_df.columns if col.startswith("g-")]

dicts = {
    "24 * D1" : D1_24,
    "48 * D1" : D1_48,
    "72 * D1" : D1_72,
    "24 * D2" : D2_24,
    "48 * D2" : D2_48,
    "72 * D2" : D2_72,
}

for val, dic in dicts.items():
    print(val)
    df = test_df[test_df["time_dose"] == val]
    for i, maxd in dic.items():
        print(i)
        vec_i = train_df[GENES].loc[i].values
        for j in df.index:
            
            vec_j = df[GENES].loc[j].values
            dis = euclidean(vec_i, vec_j)
            print(dis)
            if dis <= maxd*0.9:
                print(i, j, dis, maxd)
                ans_i = train_y[MOAS].loc[i].values
                #sub_df[MOAS].loc[j] = ans_i
                sub_df.loc[j, MOAS] = ans_i

24 * D1
11053
35.441373973084055
36.28397887952753
37.27917770927894
37.14827987282857
43.905374305544875
36.07003525324033
36.12876562989663
39.6476571239714
88.57971317497027
36.99013856989455
35.86325804134365
34.431018440208824
37.39241884847783
38.398329468350575
37.53561842223996
39.7769636644126
34.64774745665294
44.577715045076054
39.63932442890519
34.5068948931949
38.419329851130925
38.824077592262256
37.5401648308848
34.87432161247011
40.34413995501701
35.78632248415587
36.303414186684975
32.94722840391889
35.576063219108434
35.896892428314736
37.07277801244466
33.92298940379518
37.65512916934425
35.572209210702674
31.702781548469844
36.70305761677084
38.12848635351264
91.1500477860544
35.591226882477656
38.22081761226465
35.39538547791223
35.8447504132753
35.649281329362026
39.18710431123994
35.05965691375202
38.48671725218975
36.116247059322205
45.13902169542446
40.39238809602621
33.56026727783913
33.52268341064003
38.61024055325219
43.96455355988503
38.18949943793451
38.14

60.16483639660296
38.2870263226331
34.020183680280155
36.41999709980768
37.97338224862252
38.897273667700674
39.42566526604212
35.16619528041099
38.86492062065739
37.04557440180945
38.1811519951926
34.84458310139468
36.57232488535559
34.05937602569959
83.6487287993667
34.43423337029591
33.17481974193681
34.033975380199124
53.12456658289082
33.90050066577189
36.07133647524028
32.99156910984987
40.24942447551269
40.01628431163993
39.547615698294635
40.650821427247934
41.810696112119444
35.00007919962468
38.01346976217772
37.686471080481915
36.912781190801645
37.98279060153427
38.03630907383102
39.336435269734345
38.040234875194976
37.05461171635725
35.88085122109006
38.78265619204028
36.19494519957172
35.71110232966773
36.01081668860066
38.27836652340327
46.55635506093663
36.77322231896465
37.55860361887806
32.5815684337019
36.69959535308257
33.49706475737837
36.57103476427759
37.643762780306645
80.84312564763933
35.936877868423686
36.80063215774968
35.88261259022258
38.634677421456495
3

44.78242229960322
49.762749786763194
48.53305228996009
49.32627426818288
48.90999491607007
53.349047158313894
47.659221865552944
47.0981006932339
50.397141638092926
49.72999774140353
48.912660143361656
49.40214158060357
54.930795821742834
47.593281194723275
50.802137628745506
52.29508663880384
97.90338545780733
54.41539583858598
46.99089732895085
48.23722832771386
50.49483859128574
88.49270276356124
50.39026796545142
44.73885574475503
48.21621779505315
50.53681816191043
46.12164021508775
51.5444629406496
47.73557092703511
53.79358968817753
51.23734850292314
47.84704586722152
50.35934353116609
49.1147272442798
45.94920773484566
42.91444592675059
45.24693764941446
93.29437987665709
46.20087482634934
47.04050334679679
49.78856892781715
50.13084880939081
50.71487541146089
47.89249630349205
46.000504807990964
52.25609713727194
46.52172334866798
51.124189775389105
48.51471835484568
96.82469735196698
46.451575176413556
47.13234469713978
48.54500604459742
47.87279493407503
48.54010045498464
47

108.56337600747317
113.46098321586148
108.93306733609404
113.64329732782308
112.68533444982981
109.28880277242496
111.3175195440053
110.57047239353733
110.9377251481659
115.21179655265341
112.83428754088892
114.895674321621
109.1892167538077
111.38656714272147
112.53200404844837
111.249800965395
113.38969051016058
110.92513731652532
115.54706428187606
116.8269731729792
115.89201374719485
111.97406808489187
113.58449009059291
111.99309198772039
108.04630936695617
110.00453138021179
109.71078252364259
111.38502923171498
110.42975012472861
112.03555531696176
109.49275584809253
112.23663408116799
108.07527733746511
55.126109156369814
3139 854 55.126109156369814 69.80596651
117.44241836214886
108.7844120294815
103.92120485189729
109.1231540357499
110.73712136758839
109.81951692595447
109.8449833558183
109.9664142441682
111.9691263522673
109.4598951668601
97.62791417299665
110.77779900151474
110.49255254405158
115.51289537120952
109.76217779695335
108.29151449070237
112.73005198291182
109.41

112.0024739216505
108.90772469118065
110.53706519914485
110.96199959148177
112.79388748327632
109.1992800385149
110.24010983280995
52.32578628496661
3139 3426 52.32578628496661 69.80596651
64.55486049950073
114.18770335833014
110.48593570409766
109.9678095124205
111.79011866028232
103.19333032207072
112.09757651470437
99.01710105118207
57.261314434616324
3139 3464 57.261314434616324 69.80596651
109.79538017967786
115.99839043322972
109.61690883349156
112.56724318650608
109.24489942290212
112.66799085361379
113.10922806296576
55.58691507441297
3139 3527 55.58691507441297 69.80596651
116.24455130319012
108.9047919231748
111.61785305066569
109.17264579486017
113.15225540810046
112.76120028343082
113.2974630150649
109.67102583900636
109.83124655101571
108.64487665154763
111.107013725687
109.57786351170569
92.08457999057171
114.07113625413749
108.84104626642468
112.90854724470597
109.57058277740427
112.02058406480481
112.8441695659993
113.23502307696148
112.00042196880331
107.7474863368515


KeyboardInterrupt: 

In [397]:
from scipy.spatial.distance import euclidean
#train_df = pd.read_csv("../input/lish-moa/train_features.csv")
train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
#test_df = pd.read_csv("../input/lish-moa/test_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
#train_y = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
train_y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")

sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv")
MOAS = [col for col in train_y.columns if col != "sig_id"]
GENES = [col for col in train_df.columns if col.startswith("g-")]

dicts = {
    "24 * D1" : D1_24,
    "48 * D1" : D1_48,
    "72 * D1" : D1_72,
    "24 * D2" : D2_24,
    "48 * D2" : D2_48,
    "72 * D2" : D2_72,
}

ratio = 0.9
for val, dic in dicts.items():
    print(val)
    df = test_df[test_df["time_dose"] == val]
    for i, maxd in dic.items():
        print(i)
        vec_i = train_df[GENES].loc[i].values
        ans_i = train_y[MOAS].loc[i].values
        
        mat = df[GENES].values
        ds = np.sqrt(np.sum((mat - vec_i)**2, axis=1))
        target_idxs = np.where(ds <= maxd*ratio)
        target_idxs = df.iloc[target_idxs].index
        for t in target_idxs:
            print("   ", t)
        sub_df.loc[target_idxs, MOAS] = ans_i
        
        
    

24 * D1
11053
7898
    3610
    3786
3139
    32
    696
    854
    1079
    1292
    1358
    1638
    1703
    1767
    1998
    2315
    2565
    3189
    3403
    3426
    3464
    3527
12147
    681
19015
13507
    2806
5106
1036
    1969
    2117
    2705
    3456
48 * D1
19541
5150
    3511
7983
    1145
    1235
    1632
    3150
12911
    1675
    2374
22073
22923
    3750
23116
17188
    483
    3793
    3803
72 * D1
9289
19870
    1746
5954
    1484
    3570
    3946
6657
    118
2659
    2803
6618
22746
13970
15314
    1405
    1440
    3007
    3164
24 * D2
17608
4612
17419
14485
    286
    564
    636
    714
    883
    1659
    1702
    1930
    1940
    2176
    2350
    2351
    2416
    2573
    2767
    3022
    3439
    3722
    3848
    3883
14102
    1811
18273
22754
    3922
12561
9874
17611
7592
    1512
    3851
48 * D2
7061
18509
    1351
19921
    229
    840
    948
    1347
    1519
    1698
    1810
    2770
    2983
    3503
    3571
    3899
13303
   

In [398]:
sub_df.sum(axis=1)[:100]

0     103.0
1     103.0
2     103.0
3     103.0
4     103.0
5     103.0
6     103.0
7     103.0
8     103.0
9     103.0
10    103.0
11    103.0
12    103.0
13    103.0
14    103.0
15    103.0
16    103.0
17    103.0
18    103.0
19    103.0
20    103.0
21    103.0
22    103.0
23      1.0
24    103.0
25    103.0
26    103.0
27    103.0
28    103.0
29    103.0
30    103.0
31    103.0
32      2.0
33    103.0
34    103.0
35    103.0
36    103.0
37    103.0
38    103.0
39    103.0
40    103.0
41    103.0
42    103.0
43    103.0
44    103.0
45    103.0
46    103.0
47    103.0
48    103.0
49    103.0
50    103.0
51    103.0
52    103.0
53    103.0
54    103.0
55    103.0
56    103.0
57    103.0
58    103.0
59    103.0
60    103.0
61    103.0
62    103.0
63    103.0
64    103.0
65    103.0
66    103.0
67    103.0
68    103.0
69    103.0
70    103.0
71    103.0
72    103.0
73    103.0
74    103.0
75    103.0
76    103.0
77    103.0
78    103.0
79    103.0
80    103.0
81    103.0
82    103.0
83  